<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/relato_integrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/relato integrado completo2.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [7]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL' para números (float) e remove as linhas onde 'PL' seja negativo
dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abrilt+1'] != '-']
dados = dados[dados['PL'] != '-']
dados = dados[dados['LL'] != '-']
dados = dados[dados['AT'] != '-']
dados = dados[dados['AT'] != '0']
dados = dados[dados['at-1'] != '-']
dados = dados[dados['at-1'] != '0']

#transforma as variáveis em números
dados['at-1'] = pd.to_numeric(dados['at-1'], errors='coerce')
dados['AT'] = pd.to_numeric(dados['AT'], errors='coerce')
dados['LL'] = pd.to_numeric(dados['LL'], errors='coerce')
dados['VM30Abrilt+1'] = pd.to_numeric(dados['VM30Abrilt+1'], errors='coerce')
dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abrilt+1', 'PL', 'LL', 'at-1', 'AT']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")


Tipo de dados da coluna VM30Abrilt+1: float64
Tipo de dados da coluna PL: float64
Tipo de dados da coluna LL: float64
Tipo de dados da coluna at-1: float64
Tipo de dados da coluna AT: float64


<ipython-input-7-513d05fd18a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
